## **Desafío de exploración de datos de vuelos**

Una parte importante del rol de un científico de datos es explorar, analizar y visualizar datos. En este desafío, explorará un conjunto de datos del mundo real que contiene datos de vuelos del Departamento de Transporte de EE. UU.

Comencemos cargando los paquetes requeridos.


In [ ]:
# Cargue los paquetes que necesitará en esta aventura
suppressPackageStartupMessages({
  library(tidyverse)
  library(summarytools)
  library(glue)
  library(patchwork)
  })


Ahora, podemos importar los datos a R y comenzar a hacer ciencia de datos con ellos


In [ ]:
# Cargar y ver los datos
df_flights <- read_csv("https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/flights.csv", show_col_types = FALSE)

df_flights %>%
  slice_head(n = 7)


El conjunto de datos contiene los registros de vuelos nacionales de EE. UU. en 2013 y consta de los siguientes campos:

-   **Year**: Año del vuelo (todos los registros son de 2013)

-   **Month**: Mes del vuelo

-   **DayofMonth**: Día del mes en que salió el vuelo

-   **DayOfWeek**: Día de la semana en que salió el vuelo - del 1 (lunes) al 7 (domingo)

-   **Carrier**: Abreviatura de dos letras de la aerolínea.

-   **OriginAirportID**: Identificador numérico único para el aeropuerto de origen

-   **OriginAirportName**: Nombre completo del aeropuerto de origen.

-   **OriginCity**: Ciudad del aeropuerto de origen

-   **OriginState**: Estado del aeropuerto de origen

-   **DestAirportID**: Identificador numérico único para el aeropuerto de destino

-   **DestAirportName**: Nombre completo del aeropuerto de destino

-   **DestCity**: Ciudad del aeropuerto de destino

-   **DestState**: Estado del aeropuerto de destino

-   **CRSDepTime**: Hora programada de partida

-   **DepDelay**: Minutos de retraso de partida (un vuelo que partió antes de lo previsto tiene un valor negativo)

-   **DelDelay15**: Valor binario que indica una partida atrasada por más de 15 minutos (y por lo tanto se consideró "tarde")

-   **CRSArrTime**: Hora programada de arribo

-   **ArrDelay**: Minutos de retraso de arribo (un vuelo que llegó antes de lo previsto tiene un valor negativo)

-   **ArrDelay15**: Valor binario que indica un arribo atrasado por más de 15 minutos (y por lo tanto se consideró "tarde")

-   **Cancelled**: Valor binario que indica un vuelo cancelado

Su reto es explorar los datos de vuelo para analizar los posibles factores que inciden en los retrasos en la salida o llegada de un vuelo.

1.  Comience por limpiar los datos.

    -   Identifique cualquier dato nulo o faltante y reemplace por valores apropiados.

    -   Identifique y elimine cualquier valor atípico en las columnas **DepDelay** y **ArrDelay**.

2.  Explore los datos limpios.

    - Observe las estadísticas de resumen para los campos numéricos en el conjunto de datos.

    - Determine la distribución de las columnas **DepDelay** y **ArrDelay**.

    - Use estadísticas, funciones agregadas y visualizaciones para responder las siguientes preguntas:

         - *¿Cuáles son los retrasos promedio de partida y arribo?*

         - *¿Cómo se comparan los transportistas en términos de rendimiento de retrasos en el arribo?*

         - *¿Hay una diferencia notable en los retrasos de arribo para los diferentes días de la semana?*

         - *¿Qué aeropuerto de origen tiene el promedio de retraso de partida más alto?*

         - *¿Las salidas **atrasadas** tienden a provocar retrasos de arribo más prolongados que las salidas puntuales?*

         - *¿Qué ruta (del aeropuerto de origen al aeropuerto de destino) tiene más llegadas **atrasadas**?*

         - *¿Qué ruta tiene el promedio de retraso de arribo más alto?*


A veces, cuando tenemos muchas columnas en nuestros datos, puede ser difícil controlar los datos a primera vista usando `slice_head`

`glimpse` produce una versión transpuesta donde las columnas corren hacia abajo de la página y los datos se cruzan. Esto hace posible ver cada columna en un marco de datos. Además, también muestra la dimensión del tibble y los tipos de datos subyacentes de las columnas.


In [ ]:
# Eche un vistazo a sus datos
df_flights %>%
  glimpse()


## Limpiar valores faltantes

Una vez que haya importado sus datos, siempre es una buena idea limpiarlos. Lamentablemente, esto a menudo se subestima de forma frecuente, pero es un paso fundamental necesario para las operaciones posteriores en el análisis de datos.

Busquemos cuántos valores nulos hay para cada columna.


In [ ]:
# Encuentre cuántos valores nulos hay para cada columna.
colSums(is.na(df_flights))


Hmm, parece que hay algunos indicadores de "salida tardía" NA (valores faltantes) (columna DepDel15). Las salidas se consideran atrasadas si el retraso es de 15 minutos o más, así que veamos los retrasos para las que tienen un indicador de retraso NA:

**Pregunta 1.**

Comenzando con `df_flights`, **seleccione** las columnas `DepDelay` y `DepDel15` y luego **filtre** para obtener filas donde el valor de `DepDel15` sea `NA`. Asigne los resultados en una variable llamada `flights_depdel`.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Seleccione las columnas DepDelay y DepDel15
# y luego filtre el tibble para obtener las
# observaciones donde falta un valor de DepDel15

flights_depdel <- df_flights %>%
  select(...., ....) %>%
  filter(is.na(....))


Verifique su respuesta


In [ ]:
. <- ottr::check("tests/Question 1.R")


¡Buen trabajo! Ahora, echemos un vistazo (`glimpse`) a `flights_depdel`.



In [ ]:
flights_depdel %>%
  glimpse()


Desde las primeras observaciones, pareciera que los registros en `DepDel15` (que es un valor binario que indica una salida atrasada por más de 15 minutos) tienen todos un atraso correspondiente de 0 en `DepDelay` (minutos de retraso de la partida). Comprobemos mirando las estadísticas resumidas de los registros `DepDelay`:


In [ ]:
# Obtenga estadísticas de resumen usando la función summary
df_flights %>%
  filter(rowSums(is.na(.)) > 0) %>%
  select(DepDelay) %>%
  summary()


El mínimo, el máximo y la media son todos 0; así que parece que ninguno de estos fueron en realidad salidas *atrasadas*.

**Pregunta 2.**

Comenzando con `df_flights`, reemplace los valores que faltan en la columna **DepDel15** con un `0`. Asigne esto a un nombre de variable `df_flights`.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Reemplace los valores faltantes en DepDel15 por 0
df_flights <- df_flights %>%
  mutate(DepDel15 = ....)


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 2.R")


¡Buen trabajo! Ahora no faltan valores. Llevemos esto un poco más lejos.

### **Limpiar valores atípicos**

Un valor atípico es un punto de datos que difiere significativamente de otras observaciones. Vamos a crear una función que muestre la distribución y las estadísticas de resumen para una columna determinada.


In [ ]:
# Función para mostrar estadísticas de resumen y distribución para una columna
show_distribution <- function(var_data, binwidth) {

  # Obtenga estadísticas resumidas extrayendo primero los valores de la columna
  min_val <- min(pull(var_data))
  max_val <- max(pull(var_data))
  mean_val <- mean(pull(var_data))
  med_val <- median(pull(var_data))
  mod_val <- statip::mfv(pull(var_data))

  # Imprima las estadísticas
  stats <- glue::glue(
  "Minimum: {format(round(min_val, 2), nsmall = 2)}
   Mean: {format(round(mean_val, 2), nsmall = 2)}
   Median: {format(round(med_val, 2), nsmall = 2)}
   Mode: {format(round(mod_val, 2), nsmall = 2)}
   Maximum: {format(round(max_val, 2), nsmall = 2)}"
  )

  theme_set(theme_light())
  # Grafique el histograma
  hist_gram <- ggplot(var_data) +
  geom_histogram(aes(x = pull(var_data)), binwidth = binwidth,
                 fill = "midnightblue", alpha = 0.7, boundary = 0.4) +

  # Agregue líneas para las estadísticas
  geom_vline(xintercept = min_val, color = "gray33",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mean_val, color = "cyan",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = med_val, color = "red",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mod_val, color = "yellow",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = max_val, color = "gray33",
 linetype = "dashed", size = 1.3) +

  # Añada títulos y etiquetas
  ggtitle("Data Distribution") +
  xlab("") +
  ylab("Frequency") +
  theme(plot.title = element_text(hjust = 0.5))

  # Trace el diagrama de caja
  bx_plt <- ggplot(data = var_data) +
  geom_boxplot(mapping = aes(x = pull(var_data), y = 1),
               fill = "#E69F00", color = "gray23", alpha = 0.7) +

    # Añada títulos y etiquetas
  xlab("Value") +
  ylab("") +
  theme(plot.title = element_text(hjust = 0.5))


  # Para devolver múltiples salidas, use una lista (`list`)
  return(

    list(stats,
         hist_gram / bx_plt)) # Fin de las salidas devueltas 

} # Fin de la función


**Pregunta 3.** Comenzando con los datos `df_flights`, mantenga solo la columna **`DepDelay`**. Asigne esto a una variable llamada `df_col`.

Una vez resuelto, llame a la función `show_distribution` con los argumentos y los valores correspondientes de la siguiente manera: `var_data = df_col` y `binwidth = 100`

De la salida de la función, ¿cuál es la distribución de **DepDelay** (minutos de atraso de partida)?

Rellene el marcador `....` con el código correcto.

In [ ]:
# Seleccione la columna DepDelay 
df_col <- df_flights %>%
  ....

# Llame a la función show_distribution
show_distribution(var_data = df_col, binwidth = 100)


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 3.R")


Ahora, investiguemos la distribución de **ArrDelay** (minutos de atraso de arribo)

**Pregunta 4.** Comenzando con los datos `df_flights`, mantenga solo la columna **`ArrDelay`**. Asigne esto a un nombre de variable `df_col`.

Una vez que haya resuelto esto, llame a la función `show_distribution` con los argumentos y los valores correspondientes de la siguiente manera: `var_data = df_col` y `binwidth = 100` (valor del ancho de cada contenedor a lo largo del eje X)

De la salida de la función, ¿cuál es la distribución de **ArrDelay**?

Rellene el marcador `....` con el código correcto.

In [ ]:
# Seleccione la columna DepDelay 
df_col <- df_flights %>%
  ....

# Llame a la función show_distribution
show_distribution(var_data = df_col, binwidth = 100)


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 4.R")


De ambas salidas, hay valores atípicos en los extremos inferior y superior de ambas variables. Recortemos los datos para incluir solo las filas donde los valores de estos campos están dentro del percentil 1 y 90. Comenzamos con el registro **ArrDelay**.


In [ ]:
# Recorte los valores atípicos para ArrDelay en función de los percentiles del 1 % y el 90 %
# Produzca cuantiles correspondientes al 1% y 90%
arrdelay_01pcntile <- df_flights %>%
  pull(ArrDelay) %>%
  quantile(probs = 1 / 100, names = FALSE)

arrdelay_90pcntile <- df_flights %>%
  pull(ArrDelay) %>%
  quantile(probs = 90 / 100, names = FALSE)

# Imprima los cuantiles 1 y 90 respectivamente
cat(arrdelay_01pcntile, "\n", arrdelay_90pcntile)


Ahora que tenemos los cuantiles correspondientes al 1 % y al 90 %, filtremos los datos de `df_flights` para incluir solo filas cuyo retraso de llegada se encuentre dentro de este rango.

**Pregunta 5.** Comenzando con los datos de `df_flights`, filtre para incluir solo filas cuyo **ArrDelay** se encuentre dentro de los cuantiles 1 y 90. Asigne esto a una variable llamada `df_flights`.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Filtre los datos para remover los valores atípicos
df_flights <- df_flights %>%
  filter(ArrDelay > ...., ....)


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 5.R")


Ahora, hagamos lo mismo para la columna `DepDelay`.

**Pregunta 6.** A partir de los datos de `df_flights`, obtenga los cuantiles correspondientes al 1% y al 90%. Asigne estos valores a las variables `depdelay_01pcntile` y `depdelay_90pcntile` respectivamente.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Recorte los valores atípicos para DepDelay en función de los percentiles 1% y 90%
# Produzca cuantiles correspondientes al 1% y 90%
depdelay_01pcntile <- df_flights %>%
  .... %>%
  ....

depdelay_90pcntile <- df_flights %>%
  .... %>%
  ....

# Imprima los cuantiles 1 y 90 respectivamente
cat(depdelay_01pcntile, "\n", depdelay_90pcntile)


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 6.R")


¡¡Buen trabajo!!

Ahora que tenemos los cuantiles correspondientes al 1 % y al 90 %, filtremos los datos de `df_flights` para incluir solo las filas cuyo retraso en la salida se encuentre dentro de este rango.

**Pregunta 7.** Comenzando con los datos de `df_flights`, filtre para incluir solo filas cuyo **DepDelay** se encuentre dentro de los cuantiles 1 y 90. Asigne esto a una variable llamada `df_flights`.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Filtre los datos para remover los valores atípicos
df_flights <- df_flights %>%
  ....


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 7.R")


¡¡Eres genial!!

Ahora, podemos verificar la distribución de nuestras dos variables con los valores atípicos eliminados.

In [ ]:
# Distribución de DepDelay
show_distribution(var_data = select(df_flights, DepDelay), binwidth = 2)


In [ ]:
# Distribución de ArrDelay
show_distribution(var_data = select(df_flights, ArrDelay), binwidth = 2)


¡Mucho mejor!

Ahora que todos los datos están limpios, podemos comenzar a hacer un análisis exploratorio.

## **Explore los datos**

Comencemos con una vista general de las estadísticas de resumen para las columnas numéricas.


In [ ]:
# Obtenga estadísticas de resumen comunes utilizando el paquete summarytools
df_flights %>%
  descr(stats = "common")


### **¿Cuáles son los atrasos promedios de partida y arribo?**

**Pregunta 8.** Comenzando con los datos `df_flights`, use `across()` dentro de `summarize()` para encontrar la media entre las columnas `DepDelay` y `ArrDelay`. Asigne esto a una variable llamada `df_delays`. ¿Cuáles son los atrasos promedios?

Rellene el marcador `....` con el código correcto.

In [ ]:
# Resuma los atrasos de salida y llegada encontrando la media
df_delays <- df_flights %>%
  summarise(across(....))

df_delays


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 8.R")


### **¿Cómo se comparan los transportistas en términos de rendimiento de atrasos en el arribo?**

Un diagrama de caja puede ser una buena manera de representar gráficamente la distribución de grupos de datos numéricos a través de sus cuantiles. El `geom` que se ocupa de los diagramas de caja es `geom_boxplot`


In [ ]:
# Compara el retraso de llegada entre diferentes operadores
df_flights %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = Carrier, y = ArrDelay))


### **¿Cómo se comparan los transportistas en términos de rendimiento de atrasos en el arribo?**

Hagamos lo mismo con el rendimiento del retraso de arribo.

También podemos probar y reorganizar los niveles de 'Carrier' en orden ascendente del tiempo de atraso y también añadir algo de color en las tramas.


In [ ]:
df_flights %>%
  mutate(Carrier = fct_reorder(Carrier, DepDelay)) %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = Carrier, y = DepDelay, color = Carrier),
  show.legend = FALSE)


Alternativamente, para crear los gráficos anteriores, podemos usar `purr::map()` que nos permite aplicar una función a cada columna. Ver `?map` para más detalles.

In [ ]:
map(df_flights %>% select(ArrDelay, DepDelay), ~ ggplot(df_flights) +
  geom_boxplot(mapping = aes(x = Carrier, y = .x)) + ylab(""))


### **¿Algunos días de la semana son más propensos a sufrir atrasos en la llegada que otros?**

Nuevamente, utilicemos un diagrama de caja para inspeccionar visualmente la distribución de los atrasos en las llegadas según el día de la semana. Para lograr esto con éxito, primero tenemos que codificar los días de la semana en variables "categóricas", es decir, "factores".


In [ ]:
# Codifique el día de la semana como categórico y haga diagramas de caja
df_flights %>%
  mutate(DayOfWeek = factor(DayOfWeek)) %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = DayOfWeek, y = ArrDelay),
  show.legend = FALSE)


### **¿Algunos días de la semana son más propensos a sufrir atrasos en la partida que otros?**

Ahora es su turno.

**Pregunta 9.** Investiguemos si algunos días de la semana (eje x) son más propensos a atrasos en las partidas (eje y) que otros. Comience codificando el día de la semana como una variable categórica.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Codifique el día de la semana como una variable categórica
df_flights <- df_flights %>%
  mutate(....)

# Haz un diagrama de caja de DayOfWeek y DepDelay
dep_delay_plot <- df_flights %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = ...., y = ...., color = DayOfWeek),
  show.legend = FALSE) +
  scale_color_brewer(palette = "Dark2")

dep_delay_plot


¿Qué puedes hacer con esto?
Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 9.R")


¡Qué gran progreso está teniendo!

### **¿Qué aeropuerto de salida tiene el retraso promedio de partida más alto?**

Para responder esto, primero tenemos que **agrupar** los datos **por** `OriginAirportName`, luego **resumir** las observaciones por el **promedio** de su demora de salida `DepDelay` y finalmente **organizar** los retrasos promedios de salida en forma **descendente**.

Escribamos esto en código.


In [ ]:
# Utilice group_by %>% summarize para encontrar 
# los aeropuertos con el promedio más alto de DepDelay
mean_departure_delays <- df_flights %>%
  group_by(OriginAirportName) %>%
  summarize(mean_dep_delay_time = mean(DepDelay)) %>%
  arrange(desc(mean_dep_delay_time))

# Imprima las primeras 7 filas
mean_departure_delays %>%
  slice_head(n = 7)


¡Fantástico!

Representemos esto usando diagramas de barras.


In [ ]:
mean_departure_delays %>%
  # Ordene los niveles de los factores en orden descendente de tiempo de atraso
  mutate(OriginAirportName = fct_reorder(OriginAirportName,
 desc(mean_dep_delay_time))) %>%
  ggplot() +
  geom_col(mapping = aes(x = OriginAirportName, y = mean_dep_delay_time),
 fill = "midnightblue", alpha = 0.7) +
  theme(
    # Gire los marcadores X para que podamos leerlos.
    axis.text.x = element_text(angle = 90)
  )


¿Podría intentar adivinar por qué el aeropuerto de Chicago tiene la mayor cantidad de atrasos en la partida o por qué Long Beach tiene la menor cantidad?

### **¿Las salidas atrasadas tienden a provocar retrasos de arribo más prolongados que las salidas puntuales?**

**Pregunta 10.** Comenzando con los datos `df_flights`, primero codifique la columna `DepDel15` (valor binario que indica la salida atrasada por más de 15 minutos) como categórica.

Utilice un **diagrama de caja** para investigar si las salidas atrasadas (eje x) tienden a provocar retrasos de llegada más prolongados (eje y) que las salidas a tiempo. Asigne la estética de relleno a la variable `DepDel15`.

> Puede colorear un diagrama de caja usando la estética `color` (como en los ejercicios anteriores) o, más útil, con la estética `relleno`.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Encode DepDel15 as a categorical variable
df_flights <- df_flights %>%
  mutate(DepDel15 = factor(DepDel15))

arr_delay_plot <- df_flights %>%
  ggplot() +
  geom_boxplot(mapping <- aes(x = ...., y = ...., fill = ....))

arr_delay_plot


¿Esto le sorprende?
Verifique su respuesta:



In [ ]:
. <- ottr::check("tests/Question 10.R")


### **¿Qué ruta (del aeropuerto de origen al aeropuerto de destino) tiene más llegadas atrasadas?**

Finalmente, investiguemos las rutas de viaje. Comenzaremos agregando una columna `Ruta` que indica los aeropuertos de origen y destino.


In [ ]:
# Add a "Route" column
df_flights <- df_flights %>%
  mutate(Route = paste(OriginAirportName, DestAirportName, sep = ">"))


¡Excelente! Ahora podemos usar `group_by()`, `summarize()` y `arrange()` para encontrar las rutas con las llegadas más atrasadas.


In [ ]:
# Haz resúmenes agrupados para encontrar el atraso total
# asociado con una ruta en particular
df_flights %>%
  group_by(Route) %>%
  summarize(ArrDel15 = sum(ArrDel15)) %>%
  arrange(desc(ArrDel15))


### **¿Qué ruta tiene el promedio de atraso de arribo más alto?**

¡Tu turno!

**Pregunta 11.** Comenzando con los datos de `df_flights`, agrupe las observaciones por `Route` y luego cree un resumen tibble con un nombre de columna `ArrDelay` que represente el tiempo medio de atraso de llegada. Ordene esto en forma descendente.

Asigne sus resultados a una variable llamada `df_route_arrdelay`.

Rellene el marcador `....` con el código correcto.

In [ ]:
# Cree resúmenes agrupados del tiempo de atraso de arribo
df_route_arrdelay <- df_flights %>%
  .... %>%
  summarise(ArrDelay = ....) %>%
  ....(desc(....))


# Imprima las primeras 5 columnas
df_route_arrdelay %>%
  slice_head(n = 5)


Verifique su respuesta:


In [ ]:
. <- ottr::check("tests/Question 11.R")


¡Felicitaciones por terminar el primer desafío! Vamos a dejarlo hasta aquí por ahora. Por supuesto, hay otras formas de abordar este desafío. Así que siéntase libre de experimentar, buscar en el navegador y compartir sus soluciones con sus amistades.

¡Nos vemos en el próximo módulo donde comenzaremos con Machine Learning!

Feliz aprendizaje,

[Eric,](https://twitter.com/ericntay) Gold Microsoft Learn Student Ambassador.
